In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from scipy.signal import find_peaks
from skimage.restoration import denoise_tv_chambolle

import warnings
warnings.filterwarnings("ignore")

In [2]:
path = r'C:\Users\ecath\Desktop\Research\Raw Data'


mem_spectrum = pd.read_csv(path + '\LD dataset spectrum\Spectrum of electrode LD.csv', index_col=0)
niom_spectrum = pd.read_csv(path + '\LD dataset spectrum\Spectrum of optical LD.csv', index_col=0)

In [3]:
"""
Scaling of dataframe from 0 to 1 

Parameters: 
-----------
df: pd.DataFrame
    Dataframe to scale

Returns: 
-------
df: pd.DataFrame
    Scaled df

"""


def normalization(df):  
        
    y_col = [col for col in df.columns if '_yf' in col] 
    target = pd.DataFrame(df.transpose().target).transpose()
    df_ = df.iloc[:-1]
    for col in y_col:
        df_[col] = (df_[col] - df_[col].min()) / (df_[col].max() - df_[col].min())
    df = pd.concat([df_, target], axis=0)

    return(df)

In [4]:
"""
Split dataframe to driver, nondriver, noise dataframes

Parameters: 
-----------
df: pd.DataFrame
    Dataframe with all three classes 

normalize: bool, default False 
    Scale dataframe or not

Returns: 
-------
df, drivers, nondrivers, noises: pd.DataFrame
    Original dataframe, driver, nondriver, noise pd.Dataframes

"""

def make_subdf(df, normalize=False):
    
    drivers = df.transpose()[df.transpose().target == 1].transpose()
    nondrivers = df.transpose()[df.transpose().target == 0].transpose()
    noises = df.transpose()[df.transpose().target == -1].transpose()

    
    if normalize == True: 
        drivers = normalization(drivers)
        nondrivers = normalization(nondrivers)
        noises = normalization(noises)
        df = normalization(df) 
    
    drivers = drivers.drop(['target'])
    nondrivers = nondrivers.drop(['target'])
    noises = noises.drop(['target'])
    df = df.drop(['target'])
    
    return(df, drivers, nondrivers, noises)



mem_spec, drivers, nondrivers, noises = make_subdf(mem_spectrum, normalize=True)
niom_spec, drivers_opt, nondrivers_opt, noises_opt = make_subdf(niom_spectrum, normalize=True)

In [ ]:
"""
Calculate the SNR value for MEM and NIOM spectrum

Parameters: 
-----------
df: pd.DataFrame
    n x m dataframe with spectrums 

Returns: 
-------
snr: pd.DataFrame
    1 x n dataframe with correspondings SNR values
    
"""

def SNR(df):
    y_col = [col for col in df.columns if '_yf' in col]
    y_df = df[y_col]
    snr = []
        
    for i in range(y_df.shape[1]):          
        s = y_df[y_col[i]][y_df[y_col[i]] != 0]
        _, properties = find_peaks(s, height=0)
        
        num_of_avg_peak = 2 #number of highest peak tp average
        mean_max = np.mean(np.sort(properties['peak_heights'])[-num_of_avg_peak:]) 
        
        sd = s.std(axis=0)
        ratio = np.round(mean_max / sd, 2)
        ratio = np.where(sd == 0, 0, ratio)
        snr.append(ratio)
        
    snr = pd.DataFrame(snr, columns=['SNR'], index=y_df.columns)
#     snr['SNR'] = (snr['SNR'] - snr['SNR'].min()) / (snr['SNR'].max() - snr['SNR'].min()) #line to rescale SNR value from 0 to 1
    return snr

In [ ]:
print(SNR(niom_spec).describe())

In [ ]:
SNR(mem_spec).describe()

In [ ]:
SNR(drivers).describe()

In [ ]:
SNR(nondrivers).describe()

In [ ]:
SNR(noises).describe()